#  Graph RAG using LangChain and Kuzu 

Taken  from https://pypi.org/project/langchain-kuzu/ 

In [ ]:
# A default setup cell.
# It imports environment variables, define 'devtools.debug" as a buildins, set PYTHONPATH, and code auto-reload
# Copy it in other Notebooks


from dotenv import load_dotenv
from rich import print as print

load_dotenv(verbose=True)
%load_ext autoreload
%autoreload 2
%reset -f

# cSpell: disable

In [ ]:
import kuzu
from ipycytoscape import CytoscapeWidget
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_kuzu.chains.graph_qa.kuzu import KuzuQAChain
from langchain_kuzu.graphs.kuzu_graph import KuzuGraph
from langchain_openai import ChatOpenAI

from src.ai_core.llm import get_llm

In [ ]:
DB = "test_db4"
MODEL_ID = None
llm = get_llm(llm_id=MODEL_ID)

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

text = "Tim Cook is the CEO of Apple. Apple has its headquarters in California."

if True:  # ne schema
    allowed_nodes = ["Person", "Company", "Location"]
    allowed_relationships = [
        ("Person", "IS_CEO_OF", "Company"),
        ("Company", "HAS_HEADQUARTERS_IN", "Location"),
    ]
    # Define the LLMGraphTransformer
    llm_transformer = LLMGraphTransformer(
        llm=llm,
        allowed_nodes=allowed_nodes,
        allowed_relationships=allowed_relationships,
    )

    # Convert the given text into graph documents
    documents = [Document(page_content=text)]
    graph_documents = llm_transformer.convert_to_graph_documents(documents)
    db = kuzu.Database(DB)
    graph = KuzuGraph(db, allow_dangerous_requests=True)

    # Add the graph document to the graph
    graph.add_graph_documents(
        graph_documents,
        include_source=True,
    )
else:
    print("load {DB}")

In [ ]:
print(llm_transformer)

In [ ]:
llm = get_llm(llm_id="gpt_4o_azure")

In [ ]:
db = kuzu.Database(DB)
graph = KuzuGraph(db, allow_dangerous_requests=True)

# Create the KuzuQAChain with verbosity enabled to see the generated Cypher queries
chain = KuzuQAChain.from_llm(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.3),
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

# Query the graph
queries = [
    "Who is the CEO of Apple?",
    "Where is Apple headquartered?",
]

for query in queries:
    result = chain.invoke(query)
    print(f"Query: {query}\nResult: {result}\n")

In [ ]:
# Create Cytoscape widget
from src.webapp.ui_components.cypher_graph_display import get_cytoscape_json, get_cytoscape_style

cyto = CytoscapeWidget()
cyto.graph.add_graph_from_json(get_cytoscape_json(graph))
cyto.set_style(get_cytoscape_style())
# Set layout and style
cyto.set_layout(animate=True)

In [ ]:
# Display the graph
cyto